# Assignment 3 PAMAP2 Dataset
Student: Paulina Traub



## Index

1.Introduction
   -Objectives
2.Data Overview
3.Data cleansing
4.Segmentation
5.Exloratory Data Analysis
6.Hypothesis Testing
7.Model Testing
   -Training data using HR
   -Training data using HR + monitor
   -Training data using monitor
   -Training data using gyroscope
   -Training data using gyroscope + monitor
   -Training data using ankle monitor
   -Training data using chest monitor
   -Training data using hand monitor
8.Conclusion 
9.Reference

1.Introduccion 

Human activity recognition is one of the most relevant research areas for the field of machine learning as it can provide greater surveillance of chronic diseases, diseases that could be triggered and predict the long-term outcome.

Nonetheless, it is still a very challenging task, due to unsolvable problems such as sensor movement, sensor location, or unforeseen events in the environment.

For this test, data will be extracted from  the HAR hardware for everyday fitness tracking of 12 activities defined in the protocol in 9 subjects with 3 IMU and 1 HR monitors, with the aim of providing a comparative evaluation of predictive models and choosing the best fit. And for this,  were set the following objectives:

1)To identify parameters that can be associated with activity intensity. Based on how much energy they use, the dataset's activities can be categorised as light, moderate, or strong. Knowing the intensity of the activities helps users determine whether they are engaging in the prescribed amounts of physical exercise.

2)Evaluate the effectiveness of different sensing devices when used to identify physical activity. To determine which one contributes more to activity recognition, the performance of the accelerometer and gyroscope data will be compared in detail. This is helpful because it enables the construction of tracking devices that are more effective.

3)Compare the performance of different sensor positions provided in the dataset, i.e. hand, ankle and chest sensors, in order to redesign activity tracking devices more accurately.

2. Data Overview

In [1]:
# importing all the libraries
import pandas as pd
import numpy as np
import math
import os
import seaborn as sns
from sklearn.model_selection import train_test_split
from scipy import stats
%matplotlib inline
import matplotlib.pyplot as plt
from itertools import cycle, islice

from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
import warnings
warnings.filterwarnings('ignore')
from sklearn import cluster
from collections import defaultdict
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import cross_val_predict
from sklearn.metrics.cluster import silhouette_score
from sklearn.metrics import precision_score,recall_score, f1_score, confusion_matrix,roc_auc_score,roc_curve, accuracy_score
from sklearn import svm
import csv

In [2]:
subject_information = { 
"Subject ID": [101,102,103,104, 105, 106, 107, 108, 109],
"Sex":['Male','Female','Male','Male','Male','Male','Male','Male','Male'],
"Age (years)":[27,25,31,24,26,26,23,32,31],
"Height (cm)": [182,169,187,194,180,183,173,179,168],
"weight (kg)" : [83,78,92,95,73,69,86,87,65],
"resting HR (bpm)" : [75,74,68,58,70,60,60,66,54],
"Max HR (bmp)" : [193,195,189,196,194,194,197,188,189],
"Dominant Hand" : ['right','right','right','right','right','right','right','left','right'],
}
table=pd.DataFrame(subject_information)
table

,Subject ID,Sex,Age (years),Height (cm),weight (kg),resting HR (bpm),Max HR (bmp),Dominant Hand
0,101,Male,27,182,83,75,193,right
1,102,Female,25,169,78,74,195,right
2,103,Male,31,187,92,68,189,right
3,104,Male,24,194,95,58,196,right
4,105,Male,26,180,73,70,194,right
5,106,Male,26,183,69,60,194,right
6,107,Male,23,173,86,60,197,right
7,108,Male,32,179,87,66,188,left
8,109,Male,31,168,65,54,189,right


The table above provides general information on each of the subjects in the sample. However, it should be noted that there is only one woman and that, in addition, there is only one left handed person.

In addition, we were provided with information on the activity ID, the activity itself and the intensity classification.

In [3]:
clasification_cb = { 
"activityID": [1,2,3,4,5,6,7,9,10,11,12,13,16,17,18,19,20,24,0],
"activity":["lying","sitting","standing","walking","running","cycling","Nordic walking","watching TV","computer work","car driving","ascending stairs","descending stairs","vacuum cleaning","ironing","folding laundry","house cleaning","playing soccer","rope jumping","other (transient activities)"],
"intensity":["ligh","ligh","ligh","moderate","vigorous","moderate","moderate","-","-","-","moderate","moderate","light","light","-","-","-","vigorous","-"]   
}

IDactivity= pd.DataFrame(clasification_cb)
IDactivity


,activityID,activity,intensity
0,1,lying,ligh
1,2,sitting,ligh
2,3,standing,ligh
3,4,walking,moderate
4,5,running,vigorous
5,6,cycling,moderate
6,7,Nordic walking,moderate
7,9,watching TV,-
8,10,computer work,-
9,11,car driving,-


We build a maintable from each subject's.dat files, in order to visualise the data in relation to the actions performed for each participants. The table is an iteration that goes over each document and stores it in a pandas dataframe.

In [4]:
# Load data from my computer
protocol_list = ["C:/Users/ptrau/Desktop/PAMAP2_Dataset/subject101.dat",
                "C:/Users/ptrau/Desktop/PAMAP2_Dataset/subject102.dat",
                "C:/Users/ptrau/Desktop/PAMAP2_Dataset/subject103.dat",
                "C:/Users/ptrau/Desktop/PAMAP2_Dataset/subject104.dat",
                "C:/Users/ptrau/Desktop/PAMAP2_Dataset/subject105.dat",
                "C:/Users/ptrau/Desktop/PAMAP2_Dataset/subject106.dat",
                "C:/Users/ptrau/Desktop/PAMAP2_Dataset/subject107.dat",
                "C:/Users/ptrau/Desktop/PAMAP2_Dataset/subject108.dat",
                "C:/Users/ptrau/Desktop/PAMAP2_Dataset/subject109.dat"]

In [5]:
# list for the subjects.
SubjectID = [1,2,3,4,5,6,7,8,9]

# Dictionary representing the activityID and corresponding tags numbers.
activityIDdict = {0: 'transient',
              1: 'lying',
              2: 'sitting',
              3: 'standing',
              4: 'walking',
              5: 'running',
              6: 'cycling',
              7: 'Nordic_walking',
              9: 'watching_TV',
              10: 'computer_work',
              11: 'car driving',
              12: 'ascending_stairs',
              13: 'descending_stairs',
              16: 'vacuum_cleaning',
              17: 'ironing',
              18: 'folding_laundry',
              19: 'house_cleaning',
              20: 'playing_soccer',
              24: 'rope_jumping' }

main_columns = ['Timestamp','ActivityID','HeartRate']

#Hand IMU columns
IMU_hand_column = ['hand_temperature',
                   'hand_acceleration_16g (1)','hand_acceleration_16g (2)','hand_acceleration_16g (3)',
                   'hand_acceleration_6g (1)', 'hand_acceleration_6g (2)','hand_acceleration_6g (3)',
                   'hand_gyroscope (1)','hand_gyroscope (2)','hand_gyroscope (3)',
                   'hand_magnetometer (1)','hand_magnetometer (2)','hand_magnetometer (3)',
                   'hand_orientation(1)','hand_orientation(2)','hand_orientation(3)','hand_orientation(4)']
# IMU Chest columns
IMU_chest_column = ['chest_temperature',
                   'chest_acceleration_16g (1)','chest_acceleration_16g (2)','chest_acceleration_16g (3)',
                   'chest_acceleration_6g (1)','chest_acceleration_6g (2)','chest_acceleration_6g (3)',
                   'chest_gyroscope (1)','chest_gyroscope (2)','chest_gyroscope (3)',
                   'chest_magnetometer (1)','chest_magnetometer (2)','chest_magnetometer (3)',
                   'chest_orientation(1)','chest_orientation(2)','chest_orientation(3)','chest_orientation(4)']

# IMU Ankle columns
IMU_ankle_column = [ 'ankle_temperature',
                    'ankle_acceleration_16g (1)','ankle_acceleration_16g (2)','ankle_acceleration_16g (2)',
                    'ankle_acceleration_6g (1)','ankle_acceleration_6g (2)','ankle_acceleration_6g (3)',
                    'ankle_gyroscope (1)','ankle_gyroscope (2)','ankle_gyroscope (3)',
                    'ankle_magnetometer (1)','ankle_magnetometer (2)','ankle_magnetometer (3)',
                    'ankle_orientation(1)','ankle_orientation(2)','ankle_orientation(3)','ankle_orientation(4)']


#Sum up the all column headings
columnsname = IMU_hand_column + IMU_chest_column + IMU_ankle_column + main_columns
 
len(columnsname)

54

In [6]:
# creating a dataframe from subjects list
PAMAP2_Dataset=pd.DataFrame()
for subject in protocol_list:
    df = pd.read_table(subject, sep='\s+')
    df.columnsname = columnsname
    df['subject_id'] = subject[-7:-4]
    PAMAP2_Dataset = PAMAP2_Dataset.append(df, ignore_index=True)

The code reads the data files combines them into a single data frame using the pd.read_table() function.

In [7]:
#Shape Of the data frame
print("Number of rows and columns = {}".format(PAMAP2_Dataset.shape))
PAMAP2_Dataset

Number of rows and columns = (2872524, 414)


,8.38,0,104,30,2.37223,8.60074,3.51048,2.43954,8.76165,3.35465,...,0.714237,-0.235885,-0.341782,-44.2263,-10.2857,7.36341,0.466904,-0.486197,0.608846,-0.418234
0,8.39,0,NaN,30.0,2.18837,8.56560,3.66179,2.39494,8.55081,3.64207,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,8.40,0,NaN,30.0,2.37357,8.60107,3.54898,2.30514,8.53644,3.73280,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,8.41,0,NaN,30.0,2.07473,8.52853,3.66021,2.33528,8.53622,3.73277,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,8.42,0,NaN,30.0,2.22936,8.83122,3.70000,2.23055,8.59741,3.76295,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,8.43,0,NaN,30.0,2.29959,8.82929,3.54710,2.26132,8.65762,3.77788,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2872519,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,-0.037788,-0.062676,-0.127084,-46.5153,3.58240,-0.035995,0.598531,0.033615,0.799791,-0.031075
2872520,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,-0.068904,-0.027006,-0.089808,-45.7474,3.54453,0.108583,0.598428,0.033012,0.799933,-0.030018
2872521,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,-0.070888,-0.038024,-0.064709,-46.3997,4.22078,0.105504,0.598233,0.033172,0.800095,-0.029416
2872522,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.018961,-0.025796,-0.064357,-46.5282,4.48593,0.530240,0.598116,0.033427,0.800180,-0.029208


In [8]:
# Assign column names to the data frame
    PAM.columns = columns
    
# Extract the subject ID from the file name and add it as a new column in the data frame
    subject_ID = int(file[-5])
    PAM['subject_ID'] = subject_ID
    
# Concatenate the procData data frame with the dataCollection data frame
    PAMAP2 = pd.concat([PAMAP2, PAM], ignore_index=True)
    

IndentationError: unexpected indent (4073013464.py, line 2)

In [ ]:
#Reset the indices of the dataCollection data frame and drop the original indices
PAMAP2.reset_index(drop=True, inplace=True)

#Display the first few rows of the dataCollection data frame
PAMAP2.head()

In [ ]:
PAMAP2.describe(include = 'all')

## Data Cleaning

Despues de haber observado la data, se decide limpiar todos los Nan values de las columnas, especificamente hablando el la columna de heartrate

In [ ]:
df_full.count()

Eliminamos primero los valores NaN de las otras columnas, para luego eliminar los Nan values de la columna Heartrate de forma distinta.

In [ ]:
df_full= df_full.dropna(subset=['timestamp','ActivityID','hand_temperature',
                   'hand_acceleration_16g (1)','hand_acceleration_16g (2)','hand_acceleration_16g (3)',
                   'hand_acceleration_6g (1)', 'hand_acceleration_6g (2)','hand_acceleration_6g (3)',
                   'hand_gyroscope (1)','hand_gyroscope (2)','hand_gyroscope (3)',
                   'hand_magnetometer (1)','hand_magnetometer (2)','hand_magnetometer (3)',
                   'hand_orientation(1)','hand_orientation(2)','hand_orientation(3)','hand_orientation(4)','chest_temperature',
                   'chest_acceleration_16g (1)','chest_acceleration_16g (2)','chest_acceleration_16g (3)',
                   'chest_acceleration_6g (1)','chest_acceleration_6g (2)','chest_acceleration_6g (3)',
                   'chest_gyroscope (1)','chest_gyroscope (2)','chest_gyroscope (3)',
                   'chest_magnetometer (1)','chest_magnetometer (2)','chest_magnetometer (3)',
                   'chest_orientation(1)','chest_orientation(2)','chest_orientation(3)','chest_orientation(4)','ankle_temperature',
                    'ankle_acceleration_16g (1)','ankle_acceleration_16g (2)','ankle_acceleration_16g (2)',
                    'ankle_acceleration_6g (1)','ankle_acceleration_6g (2)','ankle_acceleration_6g (3)',
                    'ankle_gyroscope (1)','ankle_gyroscope (2)','ankle_gyroscope (3)',
                    'ankle_magnetometer (1)','ankle_magnetometer (2)','ankle_magnetometer (3)',
                    'ankle_orientation(1)','ankle_orientation(2)','ankle_orientation(3)','ankle_orientation(4)'])
df_full.reset_index(drop = True, inplace = True)

en este punto, tenemos que trabajar los Nan Values de HeartRate y para eso, vamos a crear otra variable para trabajar los modelos predictivos


Para manejar los valores Nulos de HeartRate, tenemos dos opciones, simplemente eliminarlos o rellenar los valores vacios con un promedio, sin embargo, esta última opción no es conveniente, ya que, la data perderá variación. Por lo que eliminaremos los valores nulos de heartrate creando una variable a parte para trabajar los modelos de predicción